# L50 - Lab 4: Analysis of results

In [ ]:
using Gadfly
using DataFrames
using Statistics

In [ ]:
latex_fonts = Theme(major_label_font="Libertine", major_label_font_size=12pt,
                    minor_label_font="Libertine", minor_label_font_size=10pt,
                    key_title_font="Libertine", key_title_font_size=10pt,
                    key_label_font="Libertine", key_label_font_size=8pt)
Gadfly.push_theme(latex_fonts)

### Exp1 - ping (RTT)

In [ ]:
# parsing function
function parse_ping(filepath)
    f = open(filepath)
    lines = readlines(f)
    close(f)
    
    lines = join(lines, " ")
    rtts = map(x -> parse(Float64, x.match), collect(eachmatch(r"(?<=time=)[\d\.]+", lines)))
    loss = parse(Int, match(r"[\d\.]+(?=% packet loss)", lines).match)
    return (rtts=rtts, loss=loss)
end;

In [ ]:
# read data
exp1dir = "data/exp1"

pings = DataFrame(src = Int[], dest = Int[], wait = Float64[], rtt = Float64[])
for idx in [(i, j) for i in 1:5, j in 1:5 if i != j]
    i, j = idx
    intervals = [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1]
    for interval in intervals
        vals = parse_ping("$exp1dir/ping-$i-$j-$interval")
        for rtt in vals[:rtts]
            push!(pings, (i, j, interval, rtt))
        end
    end
end

pings_stats = by(pings, [:src, :dest, :wait], :rtt => mean, :rtt => std);

In [ ]:
p = plot(pings,
    x=:dest,
    y=:rtt,
    xgroup=:src,
    color=:dest,
    ygroup=:wait,
    Geom.subplot_grid(Geom.violin,
        Coord.cartesian(xmin=1, xmax=5, ymin=0)),
    Scale.color_discrete,
    Guide.xlabel("dest by src"),
    Guide.ylabel("RTT /ms by wait /s"))
draw(SVG("pings_allpair_violinplot.svg", 12inch, 8inch), p)

In [ ]:
function build_ping_matrices(pingsdf)
    mats = Dict()
    for interval in [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1]
        mats[interval] = zeros(Float64, 5, 5)
        for idx in [(i, j) for i in 1:5, j in 1:5 if i != j]
            i, j = idx
            mats[interval][i, j] = pingsdf[(pingsdf.src .== i) .&
                (pingsdf.dest .== j) .&
                (pingsdf.wait .== interval), :].rtt_mean[1]
        end
    end
    return mats
end

In [ ]:
p = plot(pings_stats,
    xgroup=:wait,
    x=:src,
    y=:dest,
    color=:rtt_mean,
    Geom.subplot_grid(Geom.rectbin,
        Coord.cartesian(xmin=1, xmax=5, ymin=1, ymax=5),
        Guide.xticks(orientation=:horizontal)),
    Guide.xlabel("src by wait /s"),
    Guide.ylabel("dest"),
    Guide.colorkey(title="RTT mean"))
draw(SVG("ping_allpair_matrix_rtt_mean.svg", 10inch, 2.8inch), p)
p = plot(pings_stats,
    xgroup=:wait,
    x=:src,
    y=:dest,
    color=:rtt_std,
    Geom.subplot_grid(Geom.rectbin,
        Coord.cartesian(xmin=1, xmax=5, ymin=1, ymax=5),
        Guide.xticks(orientation=:horizontal)),
    Guide.xlabel("src by wait /s"),
    Guide.ylabel("dest"),
    Guide.colorkey(title="RTT std"))
draw(SVG("ping_allpair_matrix_rtt_std.svg", 10inch, 2.8inch), p)

In [ ]:
edges = join(["$i -- $j [len=$(build_ping_matrices(pings_stats)[0.0001][i,j])];" for i in 1:5, j in 1:5 if i != j], " ")
graph = "graph G { node [shape=box, style=filled, width=.3, height=.3]; $edges }"
f = open("ping_graph_0.1.svg", "w")
print(f, read(pipeline(`echo $graph`, `neato -Tsvg`), String))
close(f)

### Exp2 - traceroute: nothing to see here (all were 1 hop)

### Exp3 - iperf

In [ ]:
# parsing function
function parse_iperf(filepath)
    f = open(filepath)
    lines = readlines(f) # remove start
    close(f)
    
    lines = join(lines, " ")
    bandwidths = map(x -> parse(Float64, x.match), collect(eachmatch(r"[\d\.]+(?= Mbits\/sec)", lines)))
    return bandwidths
end;

In [ ]:
# read data
exp3dir = "data/exp3"

iperfs = DataFrame(src = Int[], dest = Int[], time = Int[], bandwidth = Float64[])
for idx in [(i, j) for i in 1:5, j in 1:5 if i != j]
    i, j = idx
    vals = parse_iperf("$exp3dir/iperf-$i-$j")
    for t in 1:length(vals)
        push!(iperfs, (i, j, t-1, vals[t]))
    end
end

In [ ]:
p = plot(by(iperfs, [:time, :src, :dest], :bandwidth => mean, :bandwidth => minimum, :bandwidth => maximum),
    x=:time,
    y=:bandwidth_mean,
    ymin=:bandwidth_minimum,
    ymax=:bandwidth_maximum,
    ygroup=:src,
    color=:dest,
    Geom.subplot_grid(Geom.point, Geom.line, Geom.errorbar),
    Scale.color_discrete,
    Guide.xlabel("time /s"),
    Guide.ylabel("mean bandwidth /Mbps by src"))
draw(SVG("iperfs_dest_group_by_src.svg", 10inch, 8inch), p)

In [ ]:
p = plot(iperfs,
    x=:dest,
    xgroup=:src,
    y=:bandwidth,
    Geom.subplot_grid(Geom.boxplot,
        Coord.cartesian(xmin=1, xmax=5),
        Guide.xticks(orientation=:horizontal)),
   Guide.ylabel("bandwidth /Mbps"))
draw(SVG("iperfs_boxplot_group_by_src.svg", 10inch, 8inch), p)

In [ ]:
iperfmat = zeros(Float64,5,5)
for idx in [(i, j) for i in 1:5, j in 1:5 if i != j]
    i, j = idx
    iperfmat[i, j] = mean(iperfs[(iperfs.src .== i) .& (iperfs.dest .== j), :].bandwidth) / 200
end

edges = join(["$i -- $j [len=$(iperfmat[i,j])];" for i in 1:5, j in 1:5 if i < j], " ")
graph = "graph G { node [shape=box, style=filled, width=.3, height=.3]; $edges }"
f = open("iperf_graph.svg", "w")
print(f, read(pipeline(`echo $graph`, `neato -Tsvg`), String))
close(f)

### Exp4 - iperf udp

In [ ]:
# parsing function
function parse_iperfudp(filepath)
    f = open(filepath)
    lines = readlines(f)[8:17] # remove start
    close(f)
    
    bandwidths = zeros(Int, length(lines))
    for line in enumerate(lines)
        bandwidths[line[1]] = parse(Float64, chop(match(r"[\.\d]* Mbits", line[2]).match, tail=6))
    end
    return bandwidths[1:end-1]
end;

In [ ]:
# read data
exp4dir = "data/exp4"

iperfsudp = DataFrame(src = Int[], dest = Int[], b = String[], time = Int[], bandwidth = Float64[])
for idx in [(i, j) for i in 1:5, j in 1:5 if i != j]
    i, j = idx
    bandwidths = ["150m", "200m", "250m", "300m", "350m"]
    for bandwidth in bandwidths
        vals = parse_iperf("$exp4dir/iperf-$i-$j-$bandwidth")
        for t in 1:length(vals)
            push!(iperfsudp, (i, j, bandwidth, t, vals[t]))
        end
    end
end

In [ ]:
p = plot(by(iperfsudp[(iperfsudp.time .> 0) .& (iperfsudp.time .< 10), :], [:b, :time, :dest], :bandwidth=>mean),
    x=:time,
    xgroup=:b,
    y=:bandwidth_mean,
    color=:dest,
    Geom.subplot_grid(Geom.point, Geom.line),
    Scale.color_discrete,
    Guide.xlabel("time /s by bandwidth /Mbps"),
    Guide.ylabel("mean bandwidth"))
draw(SVG("iperfs_udp_timeline.svg", 12inch, 8inch), p)

In [ ]:
p = plot(iperfsudp,
    x=:dest,
    xgroup=:src,
    y=:bandwidth,
    color=:b,
    Geom.subplot_grid(Geom.boxplot))
draw(SVG("iperfs_udp_boxplot.svg", 12inch, 8inch), p)

### Exp5 - 2 way iperf

In [ ]:
# parsing function
function parse_iperf2way(filepath)
    f = open(filepath)
    lines = readlines(f)
    close(f)
    
    lines = join(lines, "\n")
    
    ids = unique(map(x -> x.match, collect(eachmatch(r"(?<=\[)[ ]*\d(?=\])", lines))))
       
    lines1 = join(map(x -> x.match, collect(eachmatch(Regex("\\[$(ids[1])\\].*\\n"), lines))), " ")
    lines2 = join(map(x -> x.match, collect(eachmatch(Regex("\\[$(ids[2])\\].*\\n"), lines))), " ")
    
    vals1 = map(x -> parse(Float64, x.match), collect(eachmatch(r"[\d\.]+(?= Mbits\/sec)", lines1)))
    vals2 = map(x -> parse(Float64, x.match), collect(eachmatch(r"[\d\.]+(?= Mbits\/sec)", lines2)))
    
    return (vals1[1:end-1], vals2[1:end-1])
end;

In [ ]:
# read data
exp5dir = "data/exp5"

iperfs2way = DataFrame(src = Int[], dest = Int[], forwards = Bool[], time = Int[], bandwidth = Int[])
for idx in [(i, j) for i in 1:5, j in 1:5 if i < j]
    i, j = idx
    vals = parse_iperf2way("$exp5dir/iperf-$i-$j")
    for t in 1:length(vals[1])
        push!(iperfs2way, (i, j, true, t, vals[1][t]))
        push!(iperfs2way, (i, j, false, t, vals[2][t]))
    end
end

In [ ]:
# plots here
iperfs2waymod = copy(iperfs2way)
iperfs2waymod.srcdest = map(string,iperfs2waymod.src) .* "," .* map(string,iperfs2waymod.dest)
p = plot(iperfs2waymod,
    x=:srcdest,
    y=:bandwidth,
    color=:forwards,
    Geom.boxplot,
    Guide.xlabel("src-dest pairs"),
    Guide.ylabel("bandwidth /Mbps"))
draw(SVG("iperfs_2way_boxplot.svg", 12inch, 8inch), p)

### Exp6 - iperf 1 to 2

In [ ]:
# read data
exp6dir = "data/exp6"

src = 1
iperfs1to2 = DataFrame(src = Int[], dest = Int[], i = Int[], dests = Tuple{Int,Int}[], time = Int[], bandwidth = Float64[])
for dests in [(i, j) for i in 2:5, j in 2:5 if i <= j]
    src = 1
    for i in 1:length(dests)
        dest = dests[i]       
        vals = parse_iperf("$exp6dir/iperf-$src-$dest($i)-$dests")
        for t in 1:length(vals)
            push!(iperfs1to2, (src, dest, i, dests, t, vals[t]))
        end
    end
end

In [ ]:
iperfs1to2mod = copy(iperfs1to2)
iperfs1to2mod.dests_tuple = map(string, iperfs1to2mod.dests)
iperfs1to2_stats = by(iperfs1to2mod, [:dests_tuple, :i] ,:bandwidth=>mean)
p1 = plot(iperfs1to2_stats,
    x=:dests_tuple,
    y=:bandwidth_mean,
    color=:i,
    Geom.bar(position=:stack),
    Scale.color_discrete,
    Theme(bar_spacing=2mm),
    Guide.xlabel("destinations tuple"),
    Guide.ylabel("mean bandwidth /Mbps"),
    Guide.colorkey(title="tuple index"))
draw(SVG("iperfs_1to2.svg", 12inch, 8inch), p1)

### Exp7 - iperf 1 to 3

In [ ]:
# read data
exp7dir = "data/exp7"

src = 1
#iperfs1to3 = DataFrame(src = Int[], dest = Int[], i = Int[], dests = Tuple{Int,Int,Int}[], time = Int[], bandwidth = Float64[])
for dests in [(i,j,k) for i in 2:5, j in 2:5, k in 2:5 if i <= j && j <= k]
    src = 1
    for i in 1:length(dests)
        dest = dests[i]       
        vals = parse_iperf("$exp7dir/iperf-$src-$dest($i)-$dests")
        for t in 1:length(vals)
            push!(iperfs1to3, (src, dest, i, dests, t, vals[t]))
        end
    end
end

In [ ]:
iperfs1to3mod = copy(iperfs1to3)
iperfs1to3mod.dests_tuple = map(string, iperfs1to3mod.dests)
iperfs1to3_stats = by(iperfs1to3mod, [:dests_tuple, :i] ,:bandwidth=>mean)
p2 = plot(iperfs1to3_stats,
    x=:dests_tuple,
    y=:bandwidth_mean,
    color=:i,
    Geom.bar(position=:stack),
    Scale.color_discrete,
    Theme(bar_spacing=2mm),
    Guide.xlabel("destinations tuple"),
    Guide.ylabel("mean bandwidth /Mbps"),
    Guide.colorkey(title="tuple index"))
draw(SVG("iperfs_1to3.svg", 12inch, 8inch), p2)

### Exp8 - iperf 1 to 4

In [ ]:
# read data
exp8dir = "data/exp8"

src = 1
iperfs1to4 = DataFrame(src = Int[], dest = Int[], i = Int[], dests = Tuple{Int,Int,Int,Int}[], time = Int[], bandwidth = Float64[])
for dests in [(i,j,k,l) for i in 2:5, j in 2:5, k in 2:5, l in 2:5 if i <= j && j <= k && k <= l] 
    src = 1
    for i in 1:length(dests)
        dest = dests[i]       
        vals = parse_iperf("$exp8dir/iperf-$src-$dest($i)-$dests")
        for t in 1:length(vals)
            push!(iperfs1to4, (src, dest, i, dests, t, vals[t]))
        end
    end
end

In [ ]:
iperfs1to4mod = copy(iperfs1to4)
iperfs1to4mod.dests_tuple = map(string, iperfs1to4mod.dests)
iperfs1to4_stats = by(iperfs1to4mod, [:dests_tuple, :i] ,:bandwidth=>mean)
p3 = plot(iperfs1to4_stats,
    x=:dests_tuple,
    y=:bandwidth_mean,
    color=:i,
    Geom.bar(position=:stack),
    Scale.color_discrete,
    Theme(bar_spacing=2mm),
    Guide.xlabel("destinations tuple"),
    Guide.ylabel("mean bandwidth /Mbps"),
    Guide.colorkey(title="tuple index"))
draw(SVG("iperfs_1to4.svg", 12inch, 8inch), p3)

In [ ]:
p5 = gridstack(Array{Union{Context, Plot},2}([p1 p2; p3 p4]))
draw(SVG("iperfs-1-many.svg", 12inch, 8inch), p5)

### Exp9 - iperf 2 to 1

In [ ]:
# read data
exp9dir = "data/exp9"

src = 1
iperfs2to1 = DataFrame(srcs = Tuple{Int,Int}[], src = Int[], i = Int[], dest = Int[], time = Int[], bandwidth = Float64[])
for srcs in [(i, j) for i in 2:5, j in 2:5 if i <= j]
    dest = 1
    for i in 1:length(srcs)
        src = srcs[i]       
        vals = parse_iperf("$exp9dir/iperf-$srcs-$src($i)-$dest")
        for t in 1:length(vals)
            push!(iperfs2to1, (srcs, src, i, dest, t, vals[t]))
        end
    end
end

In [ ]:
iperfs2to1mod = copy(iperfs2to1)
iperfs2to1mod.srcs_tuple = map(string, iperfs2to1mod.srcs)
iperfs2to1_stats = by(iperfs2to1mod, [:srcs_tuple, :i] ,:bandwidth=>mean)
p1 = plot(iperfs2to1_stats, x=:srcs_tuple,
    y=:bandwidth_mean,
    color=:i,
    Geom.bar(position=:stack),
    Scale.color_discrete,
    Theme(bar_spacing=2mm),
    Guide.xlabel("destinations tuple"),
    Guide.ylabel("mean bandwidth /Mbps"),
    Guide.colorkey(title="tuple index"))
draw(SVG("iperfs_2to1.svg", 12inch, 8inch), p1)

### Exp10 - iperf 3 to 1

In [ ]:
# read data
exp10dir = "data/exp10"

src = 1
iperfs3to1 = DataFrame(srcs = Tuple{Int,Int,Int}[], src = Int[], i = Int[], dest = Int[], time = Int[], bandwidth = Float64[])
for srcs in [(i, j, k) for i in 2:5, j in 2:5, k in 2:5 if i <= j && j <= k]
    dest = 1
    for i in 1:length(srcs)
        src = srcs[i]       
        vals = parse_iperf("$exp10dir/iperf-$srcs-$src($i)-$dest")
        for t in 1:length(vals)
            push!(iperfs3to1, (srcs, src, i, dest, t, vals[t]))
        end
    end
end

In [ ]:
iperfs3to1mod = copy(iperfs3to1)
iperfs3to1mod.srcs_tuple = map(string, iperfs3to1mod.srcs)
iperfs3to1_stats = by(iperfs3to1mod, [:srcs_tuple, :i] ,:bandwidth=>mean)
p2 = plot(iperfs3to1_stats,
    x=:srcs_tuple,
    y=:bandwidth_mean,
    color=:i,
    Geom.bar(position=:stack),
    Scale.color_discrete,
    Theme(bar_spacing=2mm),
    Guide.xlabel("destinations tuple"),
    Guide.ylabel("mean bandwidth /Mbps"),
    Guide.colorkey(title="tuple index"))
draw(SVG("iperfs_3to1.svg", 12inch, 8inch), p2)

### Exp11 - iperf 4 to 1

In [ ]:
# read data
exp11dir = "data/exp11"

src = 1
iperfs4to1 = DataFrame(srcs = Tuple{Int,Int,Int,Int}[], src = Int[], i = Int[], dest = Int[], time = Int[], bandwidth = Float64[])
for srcs in [(i, j, k, l) for i in 2:5, j in 2:5, k in 2:5, l in 2:5 if i <= j && j <= k && k <= l]
    dest = 1
    for i in 1:length(srcs)
        src = srcs[i]
        vals = parse_iperf("$exp11dir/iperf-$srcs-$src($i)-$dest")
        for t in 1:length(vals)
            push!(iperfs4to1, (srcs, src, i, dest, t, vals[t]))
        end
    end
end

In [ ]:
iperfs4to1mod = copy(iperfs4to1)
iperfs4to1mod.srcs_tuple = map(string, iperfs4to1mod.srcs)
iperfs4to1_stats = by(iperfs4to1mod, [:srcs_tuple, :i] ,:bandwidth=>mean)
p3 = plot(iperfs4to1_stats,
    x=:srcs_tuple,
    y=:bandwidth_mean,
    color=:i,
    Geom.bar(position=:stack),
    Scale.color_discrete,
    Theme(bar_spacing=2mm),
    Guide.xlabel("destinations tuple"),
    Guide.ylabel("mean bandwidth /Mbps"),
    Guide.colorkey(title="tuple index"))
draw(SVG("iperfs_4to1.svg", 12inch, 8inch), p3)

In [ ]:
p5 = gridstack(Array{Union{Context, Plot},2}([p1 p2; p3 p4]))
draw(SVG("iperfs-many-1.svg", 12inch, 8inch), p5)

### Exp12 - ping with cross-talk

In [ ]:
# read data
exp12dir = "data/exp12"

pings_cross = DataFrame(src = Int[], dest = Int[], wait = Float64[], rtt = Float64[])
for idx in [(i, j) for i in 1:5, j in 1:5 if i != j]
    i, j = idx
    intervals = [0.000001, 0.00001, 0.0001, 0.001, 0.01, 0.1]
    for interval in intervals
        vals = parse_ping("$exp12dir/ping-$i-$j-$interval")
        for rtt in vals[:rtts]
            push!(pings_cross, (i, j, interval, rtt))
        end
    end
end

pings_cross_stats = by(pings_cross, [:src, :dest, :wait], :rtt => mean, :rtt => std);

In [ ]:
pings_cross_tagged = copy(pings_cross_stats)
pings_cross_tagged.t = "cross"
pings_tagged = copy(pings_stats)
pings_tagged.t = "non-cross"

pings_combined = vcat(pings_cross_tagged, pings_tagged)

p = plot(pings_combined,
    xgroup=:wait,
    ygroup=:t,
    x=:src,
    y=:dest,
    color=:rtt_mean,
    Geom.subplot_grid(Geom.rectbin,
        Coord.cartesian(xmin=1, xmax=5, ymin=1, ymax=5)))
draw(SVG("pings_combined_mean.svg", 10inch, 4inch), p)
p = plot(pings_combined,
    xgroup=:wait,
    ygroup=:t,
    x=:src,
    y=:dest,
    color=:rtt_std,
    Geom.subplot_grid(Geom.rectbin,
        Coord.cartesian(xmin=1, xmax=5, ymin=1, ymax=5)))
draw(SVG("pings_combined_std.svg", 10inch, 4inch), p)

In [ ]:
p = plot(pings_cross_stats,
    xgroup=:wait,
    x=:src,
    y=:dest,
    color=:rtt_mean,
    Geom.subplot_grid(Geom.rectbin,
        Coord.cartesian(xmin=1, xmax=5, ymin=1, ymax=5),
        Guide.xticks(orientation=:horizontal)),
    Guide.xlabel("src by wait /s"),
    Guide.ylabel("dest"),
    Guide.colorkey(title="RTT mean"))
draw(SVG("ping_allpair_matrix_rtt_mean_cross.svg", 10inch, 2.8inch), p)
p = plot(pings_cross_stats,
    xgroup=:wait,
    x=:src,
    y=:dest,
    color=:rtt_std,
    Geom.subplot_grid(Geom.rectbin,
        Coord.cartesian(xmin=1, xmax=5, ymin=1, ymax=5),
        Guide.xticks(orientation=:horizontal)),
    Guide.xlabel("src by wait /s"),
    Guide.ylabel("dest"),
    Guide.colorkey(title="RTT std"))
draw(SVG("ping_allpair_matrix_rtt_std_cross.svg", 10inch, 2.8inch), p)

In [ ]:
edges = join(["$i -- $j [len=$(build_ping_matrices(pings_cross_stats)[0.1][i,j])];" for i in 1:5, j in 1:5 if i != j], " ")
graph = "graph G { node [shape=box, style=filled, width=.3, height=.3]; $edges }"
f = open("ping_graph_0.1.svg", "w")
print(f, read(pipeline(`echo $graph`, `neato -Tsvg`), String))
close(f)

### Exp13 - iperf with cross-talk

In [ ]:
# read data
exp13dir = "data/exp13"

iperfs_cross = DataFrame(src = Int[], dest = Int[], time = Int[], bandwidth = Float64[])
for idx in [(i, j) for i in 1:5, j in 1:5 if i != j]
    i, j = idx
    vals = parse_iperf("$exp13dir/iperf-$i-$j")
    for t in 1:length(vals)
        push!(iperfs_cross, (i, j, t-1, vals[t]))
    end
end

In [ ]:
p = plot(iperfs_cross,
    x=:dest,
    xgroup=:src,
    y=:bandwidth,
    Geom.subplot_grid(Geom.boxplot,
        Coord.cartesian(xmin=1, xmax=5)))
draw(SVG("iperfs_boxplot_group2_by_src.svg", 10inch, 8inch), p)